In [3]:
from lxml import html
import requests
import time
import re
import cssselect

In [4]:
prob_url = "https://www.fangraphs.com/livescoreboard.aspx?date={date}".format(date = time.strftime("%Y-%m-%d"))
prob_page = requests.get(prob_url)
prob_tree = html.fromstring(prob_page.content)

In [5]:
prob_teams = []
for a in prob_tree.xpath('//tr//td//a[contains(@href, "winss.aspx")]'):
    prob_teams.append(re.search(r"team=([a-zA-Z ]*)&", str([a.attrib['href']])).group(1).lower())
prob_odds = prob_tree.xpath('//center/table//tr/td[@align="center"]/text()')

In [6]:
probs_dict = {}
i=0
while i < len(prob_odds):
    probs_dict[prob_teams[i]]=float(prob_odds[i][:4])
    i+=1
print(probs_dict)

{'red sox': 65.5, 'rays': 34.5, 'pirates': 54.7, 'reds': 45.3, 'angels': 47.0, 'blue jays': 53.0, 'braves': 46.4, 'phillies': 53.6, 'giants': 31.0, 'astros': 69.0, 'marlins': 41.8, 'mets': 58.2, 'orioles': 54.6, 'white sox': 45.4, 'padres': 36.7, 'nationals': 63.3, 'yankees': 54.3, 'rangers': 45.7, 'indians': 49.0, 'cubs': 51.0, 'tigers': 37.6, 'twins': 62.4, 'diamondbacks': 39.0, 'brewers': 61.0, 'mariners': 45.2, 'athletics': 54.8, 'royals': 33.7, 'cardinals': 66.3, 'rockies': 42.7, 'dodgers': 57.3}


In [7]:
betting_page = requests.get('http://www.vegasinsider.com/mlb/odds/las-vegas/')
betting_tree = html.fromstring(betting_page.content)
bet_teams = []
for a in betting_tree.xpath('//td[@class="viBodyBorderNorm"]//td//b/a[contains(@href, "/team/")]'):
    bet_teams.append(re.search(r"team\/([a-zA-Z -]+)", str([a.attrib['href']])).group(1).lower().replace('-', ' '))
bet_odds = betting_tree.xpath('//td[@class="viBodyBorderNorm"]//td[3]//a[contains(@href, "/line-movement/")]/text()')

In [8]:
total_odds = {}
i=0
j=0
while i < len(bet_teams):
    if i%3:
        if bet_teams[j] not in total_odds.keys():
            total_odds[bet_teams[j]]=bet_odds[i]
            j+=1
    i+=1

In [34]:
class Game:
    "A class holding opponents, odds, probabilities, and any other stat concerning the two teams playing a certain game"
    def __init__(self, team1, team2, team1odds, team2odds, team1winpct, team2winpct):
        self.team1 = team1
        self.team2 = team2
        self.team1odds = team1odds
        self.team2odds = team2odds
        self.team1winpct = team1winpct
        self.team2winpct = team2winpct
    
    def risk_team1(self):
        if float(self.team1odds) > 0:
            win_return =  (self.team1winpct)/100 * self.team1odds/100 
            loss_return = 1 - self.team1winpct/100
            win_total = win_return - loss_return 
        else:
            win_return =  -self.team1winpct/self.team1odds 
            loss_return = 1 - self.team1winpct/100
            win_total = win_return - loss_return 
        return win_total +1
    
    def risk_team2(self):
        if float(self.team2odds) > 0:
            win_return =  (self.team2winpct)/100 * self.team2odds/100 
            loss_return = 1 - self.team2winpct/100
            win_total = win_return - loss_return 
        else:
            win_return =  -self.team2winpct/self.team2odds 
            loss_return = 1 - self.team2winpct/100
            win_total = win_return - loss_return 
        return win_total +1

            

In [35]:
i=0
games={}
while i < len(prob_odds):
    key = 'game_'+str(int(i/2 + 1))
    if (prob_teams[i] in total_odds.keys()) and(prob_teams[i+1] in total_odds.keys()): 
        games[key]=Game(prob_teams[i], 
                        prob_teams[i+1], 
                        float(total_odds[prob_teams[i]]), 
                        float(total_odds[prob_teams[i+1]]), 
                        float(prob_odds[i][:4]), 
                        float(prob_odds[i+1][:4]))
    i+=2
print(games)

{'game_1': <__main__.Game object at 0x10ebbd978>, 'game_2': <__main__.Game object at 0x10ebbd9e8>, 'game_3': <__main__.Game object at 0x10ebbda58>, 'game_4': <__main__.Game object at 0x10ebbdac8>, 'game_6': <__main__.Game object at 0x10ebbdb70>, 'game_7': <__main__.Game object at 0x10ebbdb38>, 'game_8': <__main__.Game object at 0x10ebbdc18>, 'game_9': <__main__.Game object at 0x10ebbdc88>, 'game_12': <__main__.Game object at 0x10ebbdd68>, 'game_15': <__main__.Game object at 0x10ebbddd8>}


In [36]:
for key in games:
    print(games[key].team1, games[key].risk_team1(), games[key].team2, games[key].risk_team2())

red sox 0.9527272727272728 rays 1.0005
pirates 0.9677692307692308 reds 0.9965999999999999
angels 0.9011926605504588 blue jays 1.0547524752475248
braves 0.9372799999999999 phillies 1.0145714285714287
marlins 0.8443599999999999 mets 1.1016428571428571
orioles 0.9828000000000001 white sox 0.9760999999999999
padres 0.9909000000000001 nationals 0.9661578947368421
yankees 0.8823749999999999 rangers 1.11965
diamondbacks 0.9165 brewers 1.0306896551724138
rockies 0.9821000000000001 dodgers 0.9822857142857142


In [33]:
print(games['game_8'].team2odds)

-190.0
